In [23]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from skimage.io import imread
from sklearn.metrics import accuracy_score

In [24]:
import torch
import torchvision
import torchvision.transforms as transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [25]:
IMAGE_HEIGHT = 299
IMAGE_WIDTH = 299
BATCH_SIZE = 32

RANDOM_SEED = 42
EPOCHS = 10

PATH = './incept.pth'

In [26]:
train_triplets = np.loadtxt('./data/train_triplets.txt', dtype='str')
test_triplets = np.loadtxt('./data/test_triplets.txt', dtype='str')
len(train_triplets)

59515

In [27]:
# train_set, valid_set = train_test_split(train_triplets, test_size=0.1, random_state=RANDOM_SEED)
# len(train_set)

In [28]:
df_train = pd.DataFrame(train_triplets)
df_test = pd.DataFrame(test_triplets)

df_train

,0,1,2
0,02461,03450,02678
1,02299,02499,04987
2,04663,01056,03029
3,04532,01186,01297
4,03454,03809,02204
...,...,...,...
59510,00466,02952,02530
59511,02646,03580,02359
59512,03255,04844,04334
59513,02136,04619,00161


In [29]:
df_train.columns = ['A', 'B', 'C']
df_train.insert(df_train.shape[1], 'y', 1)
df_train_append = df_train.copy()
df_train_append['A'] = df_train['B'].copy()
df_train_append['B'] = df_train['A'].copy()
df_train = df_train.append(df_train_append).reset_index(drop=True)
df_train = df_train.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
df_train

,A,B,C,y
0,04942,01841,04526,1
1,00313,02280,00786,1
2,03414,03176,02052,1
3,01699,00046,03309,1
4,01125,00510,00700,1
...,...,...,...,...
119025,01851,00020,00973,1
119026,04187,04141,00354,1
119027,03094,00714,01913,1
119028,01682,02832,00026,1


In [30]:
train_set, valid_set = train_test_split(df_train.to_numpy(), test_size=0.1, random_state=RANDOM_SEED)
valid_set

array([['02927', '03675', '00358', 1],
       ['01953', '02746', '00994', 1],
       ['03002', '01320', '00227', 1],
       ...,
       ['02097', '00322', '00947', 1],
       ['04476', '03859', '02654', 1],
       ['04236', '03336', '00055', 1]], dtype=object)

In [31]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(IMAGE_HEIGHT, IMAGE_WIDTH)),
    transforms.RandomHorizontalFlip(), 
    transforms.Normalize((0.608, 0.516, 0.412),(0.264, 0.275, 0.296)), 
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(IMAGE_HEIGHT, IMAGE_WIDTH)),
    transforms.Normalize((0.608, 0.516, 0.412),(0.264, 0.275, 0.296)),
])

In [32]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None, trainning=False):

        self.imgs = data

        self.img_A = self.imgs[:, 0]
        self.img_B = self.imgs[:, 1]
        self.img_C = self.imgs[:, 2]
        # self.label = data[:, 3]
        self.transform = transform
        self.trainning = trainning
        
        if self.trainning:
            self.label = self.imgs[:, 3]
    
    def __getitem__(self, index):
        img_A = imread( './food/' + self.img_A[index] + '.jpg')
        img_B = imread( './food/' + self.img_B[index] + '.jpg')
        img_C = imread( './food/' + self.img_C[index] + '.jpg')

        if self.transform is not None:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)
            img_C = self.transform(img_C)

        if self.trainning:
            label = self.label[index]
            return img_A, img_B, img_C, label
        else:
            return img_A, img_B, img_C

    def __len__(self):
        return self.imgs.shape[0]


train_data = MyDataset(data=train_set, transform=transform_train, trainning=True)
valid_data = MyDataset(data=valid_set, transform=transform_test, trainning=True)
test_data = MyDataset(data=test_triplets, transform=transform_test, trainning=False)
train_data


In [33]:
from torch.utils.data import DataLoader 

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE)

In [34]:
import torch.nn as nn
import torchvision.models as models

class ConvNet(nn.Module):
    def __init__(self, pretrained_net):
        super(ConvNet, self).__init__()
        self.pretrained_net = pretrained_net
    def forward(self, img_A, img_B, img_C):
        anchor = self.pretrained_net(img_A)
        postive = self.pretrained_net(img_B)
        negative = self.pretrained_net(img_C)
        return anchor, postive, negative

model = models.inception_v3(pretrained=True)
net = ConvNet(model).to(device)

In [35]:
import torch.optim as optim

criterion = nn.TripletMarginLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [36]:
# for A, B, C, label in train_loader:
#     print(A)
#     break
# for i, data in enumerate(train_loader, 0):
#     print(data[3].reshape(-1,1))
#     print(data[3].size())
#     break
# y_pred = [0, 2, 1, 3]
# y_true = [0, 1, 2, 3]
# accuracy_score(y_true, y_pred)

In [37]:
def valid(anchor, positive, negative, label):
    dist_pos = torch.norm(anchor - positive, p=2, dim=1)
    dist_neg = torch.norm(anchor - negative, p=2, dim=1)
    diff = (dist_pos - dist_neg).detach().cpu().numpy()
    predict = np.ceil(diff.clip(0,1))
    
    return accuracy_score(label, predict, normalize=False)

In [38]:
def train(model):
    valid_accuracy = 0.0
    for epoch in range(EPOCHS):
        running_loss = 0.0
        valid_loss = 0.0
        correct = 0
        total = 0
        accuracy = 0.0

        print('Training')
        # trainning
        for i, data in tqdm(enumerate(train_loader, 0)):
            img_A = data[0].to(device)
            img_B = data[1].to(device)
            img_C = data[2].to(device)
            
            optimizer.zero_grad()

            anchor, positive, negetive = model(img_A, img_B, img_C)

            loss = criterion(anchor, positive, negetive)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'[{epoch + 1}] average loss per epoch: {running_loss / len(train_loader):.3f}')

        print('Validation')
        # validation
        model.eval()
        with torch.no_grad():
            for i, data in tqdm(enumerate(valid_loader, 0)):
                img_A = data[0].to(device)
                img_B = data[1].to(device)
                img_C = data[2].to(device)
                labels = data[3].reshape(-1, 1)

                anchor, positive, negetive = model(img_A, img_B, img_C)
                loss = criterion(anchor, positive, negetive)
                valid_loss += loss
                correct += valid(anchor, positive, negetive, labels)
                total += labels.size(0)
                accuracy = correct / total
        print(f'[{epoch + 1}] Accuracy of the network on the {total} valid images: {100 * accuracy} %')

        if accuracy >= valid_accuracy:
            valid_accuracy = accuracy
            torch.save(model.state_dict(), PATH)
    print('Finished Training')
    

In [39]:
train(net)

Training


2139it [31:49,  1.12it/s]

In [ ]:
net.load_state_dict(torch.load(PATH))
net.to(device)

In [ ]:
def predict(anchor, positive, negative):
    dist_pos = torch.norm(anchor - positive, p=2, dim=1)
    dist_neg = torch.norm(anchor - negative, p=2, dim=1)
    diff = (dist_pos - dist_neg).detach().to('cpu').numpy()
    predict = np.ceil(diff.clip(0,1))

    return(predict)

In [ ]:
def test(model):
    predictions = []
    model.eval()
    with torch.no_grad():
        for i, data in tqdm(enumerate(test_loader, 0)):
            img_A = data[0].to(device)
            img_B = data[1].to(device)
            img_C = data[2].to(device)

            anchor, positive, negetive = model(img_A, img_B, img_C)

            pred = predict(anchor, positive, negetive)
            predictions.append(pred)
    return predictions

In [ ]:
output = test(net)
np.savetxt('predictions_incept.txt', output, fmt='%i')